In [1]:
from time import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from ImagesloaderRAM import *
from Trainfunction import *
from Savemodel import *
from Mycudatransformation import *
from MynnModule import *
from PSNR import *

import torchvision.transforms as T
import torchvision.utils as tu

import numpy as np

%load_ext autoreload
%autoreload 2


dtype = torch.FloatTensor
gpu_dtype = torch.cuda.FloatTensor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')


print('using device:', device)

using device: cuda


## Train

In [3]:
# Load single frame denoiser
SFDenoiser, optimizer = Load_model('./SFD_C_25NB10000')

# Loading the dataset of images
# We do not load the dataset as it is to big but we can split it.

paths=['cl1']

## Training from scratch

In [4]:
MFDenoiser = MFD_C(SFDenoiser).to(device)

loss = nn.L1Loss().to(device)

optimizer = optim.Adam(MFDenoiser.parameters(), lr=0.00001, betas=(
    0.9, 0.999), eps=1e-08, weight_decay=0.00001, amsgrad=False)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.9, patience=500, verbose=True,
    threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)

In [5]:
##
def trainburstserveur(model, paths, loss_fn, optimizer, scheduler, name, Nb_frames=4, batch_size=10, num_epochs=100, nb_subepoch=1000, save_every=1):

    tic = time()
    model.train()

    loss_history = []
    loss_sfd_history = []
    loss_mfd_history = []

    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))

        for path in paths:
            trainset = 0
            trainloader = 0
            trainset = Burstfolder(path, 0.1, Nb_frames, MyRandomCrop(
                64), Randomnoise=False, loader=RGB_loader, loadram='cpu')
            trainloader = torch.utils.data.DataLoader(
                trainset, shuffle=False, batch_size=batch_size, drop_last=True)

            for subepoch in range(nb_subepoch):
                tac = time()
                if tac-tic > 36000:
                    Save_modelloss(model, optimizer, loss_history,
                                   name+'%s ' % int(epoch+1))
                    tic = time()

                for t, (x, y) in enumerate(trainloader):

                    x = Variable(torch.transpose(x.to(device), 0, 1))
                    y = Variable(torch.transpose(y.to(device), 0, 1))

                    mfinit1, mfinit2, mfinit3, mfinit4, mfinit5, mfinit6, mfinit7 = torch.zeros(
                        7, trainloader.batch_size, 64, 64, 64).to(device)
                    mfinit8 = torch.zeros(
                        trainloader.batch_size, 3, 64, 64).to(device)

                    i = 0
                    for frame, target in zip(x, y):
                        if i == 0:
                            i += 1
                            dframe, mf1, mf2, mf3, mf4, mf5, mf6, mf7, mf8 = model(
                                frame, mfinit1, mfinit2, mfinit3, mfinit4, mfinit5, mfinit6, mfinit7, mfinit8)
                            loss_sfd = loss_fn(dframe, target)
                            loss_mfd = loss_fn(mf8, target)

                        else:
                            dframe, mf1, mf2, mf3, mf4, mf5, mf6, mf7, mf8 = model(
                                frame, mf1, mf2, mf3, mf4, mf5, mf6, mf7, mf8)
                            loss_sfd += loss_fn(dframe, target)
                            loss_mfd += loss_fn(mf8, target)

                    loss = loss_sfd+loss_mfd

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                print('subepoch = %d, loss = %.4f,loss_sfd = %.4f,loss_mfd = %.4f' % (
                    subepoch + 1, loss.data, loss_sfd.data, loss_mfd.data))  # loss_mfd.data

                loss_sfd_history.append(loss_sfd)
                loss_mfd_history.append(loss_mfd)
                loss_history.append(loss)

                scheduler.step(loss.data)

        print('epoch = %d, loss = %.4f' % (epoch + 1, loss.data))
        if (epoch+1) % save_every == 0:
            Save_modelloss_mfd(model, optimizer, loss_history, loss_sfd_history,
                               loss_mfd_history, name+'%s ' % int(epoch+1))

In [6]:
trainburstserveur(MFDenoiser, paths, loss, optimizer, scheduler, "MFDC25NB",
                  Nb_frames=8, batch_size=13, num_epochs=10, nb_subepoch=500, save_every=1)

Starting epoch 1 / 10
subepoch = 1, loss = 4.8297,loss_sfd = 0.3730,loss_mfd = 4.4567
subepoch = 2, loss = 5.2296,loss_sfd = 0.3967,loss_mfd = 4.8329
subepoch = 3, loss = 3.8488,loss_sfd = 0.3464,loss_mfd = 3.5024
subepoch = 4, loss = 3.9679,loss_sfd = 0.3120,loss_mfd = 3.6559
subepoch = 5, loss = 3.2913,loss_sfd = 0.3157,loss_mfd = 2.9757
subepoch = 6, loss = 3.4883,loss_sfd = 0.3018,loss_mfd = 3.1865
subepoch = 7, loss = 3.2836,loss_sfd = 0.2717,loss_mfd = 3.0119
subepoch = 8, loss = 2.8733,loss_sfd = 0.3242,loss_mfd = 2.5491
subepoch = 9, loss = 2.6526,loss_sfd = 0.3000,loss_mfd = 2.3526
subepoch = 10, loss = 2.4157,loss_sfd = 0.3582,loss_mfd = 2.0575
subepoch = 11, loss = 2.2333,loss_sfd = 0.2540,loss_mfd = 1.9793
subepoch = 12, loss = 2.4027,loss_sfd = 0.3308,loss_mfd = 2.0718
subepoch = 13, loss = 2.1436,loss_sfd = 0.4589,loss_mfd = 1.6848
subepoch = 14, loss = 1.7821,loss_sfd = 0.2877,loss_mfd = 1.4944
subepoch = 15, loss = 1.7615,loss_sfd = 0.2704,loss_mfd = 1.4911
subepoch = 1

subepoch = 126, loss = 0.9338,loss_sfd = 0.3767,loss_mfd = 0.5571
subepoch = 127, loss = 0.6909,loss_sfd = 0.2569,loss_mfd = 0.4341
subepoch = 128, loss = 0.8278,loss_sfd = 0.3118,loss_mfd = 0.5160
subepoch = 129, loss = 0.8599,loss_sfd = 0.2498,loss_mfd = 0.6101
subepoch = 130, loss = 0.6798,loss_sfd = 0.2097,loss_mfd = 0.4701
subepoch = 131, loss = 0.7529,loss_sfd = 0.2303,loss_mfd = 0.5226
subepoch = 132, loss = 0.7934,loss_sfd = 0.2554,loss_mfd = 0.5380
subepoch = 133, loss = 0.7184,loss_sfd = 0.2550,loss_mfd = 0.4634
subepoch = 134, loss = 0.6928,loss_sfd = 0.2502,loss_mfd = 0.4426
subepoch = 135, loss = 0.8218,loss_sfd = 0.3002,loss_mfd = 0.5215
subepoch = 136, loss = 1.0920,loss_sfd = 0.5230,loss_mfd = 0.5690
subepoch = 137, loss = 0.7490,loss_sfd = 0.2353,loss_mfd = 0.5137
subepoch = 138, loss = 0.7673,loss_sfd = 0.2355,loss_mfd = 0.5318
subepoch = 139, loss = 0.6912,loss_sfd = 0.2733,loss_mfd = 0.4180
subepoch = 140, loss = 1.0136,loss_sfd = 0.4458,loss_mfd = 0.5678
subepoch =

subepoch = 250, loss = 0.6448,loss_sfd = 0.2453,loss_mfd = 0.3995
subepoch = 251, loss = 0.6350,loss_sfd = 0.2364,loss_mfd = 0.3986
subepoch = 252, loss = 0.7095,loss_sfd = 0.2443,loss_mfd = 0.4653
subepoch = 253, loss = 0.5709,loss_sfd = 0.2343,loss_mfd = 0.3366
subepoch = 254, loss = 0.7219,loss_sfd = 0.2852,loss_mfd = 0.4367
subepoch = 255, loss = 0.6112,loss_sfd = 0.2165,loss_mfd = 0.3947
subepoch = 256, loss = 0.5835,loss_sfd = 0.2290,loss_mfd = 0.3545
subepoch = 257, loss = 0.5469,loss_sfd = 0.1926,loss_mfd = 0.3543
subepoch = 258, loss = 0.5191,loss_sfd = 0.2047,loss_mfd = 0.3144
subepoch = 259, loss = 0.6306,loss_sfd = 0.2192,loss_mfd = 0.4114
subepoch = 260, loss = 0.6102,loss_sfd = 0.2441,loss_mfd = 0.3661
subepoch = 261, loss = 0.7667,loss_sfd = 0.4121,loss_mfd = 0.3547
subepoch = 262, loss = 0.5892,loss_sfd = 0.2139,loss_mfd = 0.3752
subepoch = 263, loss = 0.6348,loss_sfd = 0.2297,loss_mfd = 0.4051
subepoch = 264, loss = 0.6394,loss_sfd = 0.2572,loss_mfd = 0.3823
subepoch =

subepoch = 374, loss = 0.7608,loss_sfd = 0.4102,loss_mfd = 0.3506
subepoch = 375, loss = 0.5325,loss_sfd = 0.2165,loss_mfd = 0.3160
subepoch = 376, loss = 0.5149,loss_sfd = 0.2073,loss_mfd = 0.3076
subepoch = 377, loss = 0.6093,loss_sfd = 0.2353,loss_mfd = 0.3740
subepoch = 378, loss = 0.6431,loss_sfd = 0.3097,loss_mfd = 0.3334
subepoch = 379, loss = 0.5531,loss_sfd = 0.2239,loss_mfd = 0.3292
subepoch = 380, loss = 0.7225,loss_sfd = 0.2309,loss_mfd = 0.4916
subepoch = 381, loss = 0.5491,loss_sfd = 0.2362,loss_mfd = 0.3129
subepoch = 382, loss = 0.6224,loss_sfd = 0.3033,loss_mfd = 0.3191
subepoch = 383, loss = 0.5172,loss_sfd = 0.2363,loss_mfd = 0.2810
subepoch = 384, loss = 0.7443,loss_sfd = 0.3517,loss_mfd = 0.3926
subepoch = 385, loss = 0.7721,loss_sfd = 0.3665,loss_mfd = 0.4057
subepoch = 386, loss = 1.0577,loss_sfd = 0.5118,loss_mfd = 0.5459
subepoch = 387, loss = 0.6332,loss_sfd = 0.2264,loss_mfd = 0.4068
subepoch = 388, loss = 0.8254,loss_sfd = 0.4078,loss_mfd = 0.4176
subepoch =

subepoch = 498, loss = 0.6560,loss_sfd = 0.2117,loss_mfd = 0.4443
subepoch = 499, loss = 0.5107,loss_sfd = 0.2404,loss_mfd = 0.2703
subepoch = 500, loss = 0.6949,loss_sfd = 0.2345,loss_mfd = 0.4604
epoch = 1, loss = 0.6949
Starting epoch 2 / 10


KeyboardInterrupt: 

## Load from a previous training

In [3]:
filename = 'MFDCnew5'

In [4]:
MFDenoiser, optimizer, loss = Load_modelloss(filename)
loss = nn.L1Loss().to(device)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.9, patience=500, verbose=True,
    threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)


paths=['cl1']

In [5]:
def trainburstserveur(model, paths, loss_fn, optimizer, scheduler, name, Nb_frames=4, batch_size=10, num_epochs=100, nb_subepoch=1000, save_every=1):

    tic = time()
    model.train()

    loss_history = []
    loss_sfd_history = []
    loss_mfd_history = []

    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))

        for path in paths:
            trainset = 0
            trainloader = 0
            trainset = Burstfolder(path, 0.1, Nb_frames, MyRandomCrop(
                64), Randomnoise=False, loader=RGB_loader, loadram='cpu')
            trainloader = torch.utils.data.DataLoader(
                trainset, shuffle=False, batch_size=batch_size, drop_last=True)

            for subepoch in range(nb_subepoch):
                tac = time()
                if tac-tic > 36000:
                    Save_modelloss(model, optimizer, loss_history,
                                   name+'%s ' % int(epoch+1))
                    tic = time()

                for t, (x, y) in enumerate(trainloader):

                    x = Variable(torch.transpose(x.to(device), 0, 1))
                    y = Variable(torch.transpose(y.to(device), 0, 1))

                    mfinit1, mfinit2, mfinit3, mfinit4, mfinit5, mfinit6, mfinit7 = torch.zeros(
                        7, trainloader.batch_size, 64, 64, 64).to(device)
                    mfinit8 = torch.zeros(
                        trainloader.batch_size, 3, 64, 64).to(device)

                    i = 0
                    for frame, target in zip(x, y):
                        if i == 0:
                            i += 1
                            dframe, mf1, mf2, mf3, mf4, mf5, mf6, mf7, mf8 = model(
                                frame, mfinit1, mfinit2, mfinit3, mfinit4, mfinit5, mfinit6, mfinit7, mfinit8)
                            loss_sfd = 0
                            loss_sfd = loss_fn(dframe, target)
                            loss_mfd = loss_fn(mf8, target)

                        else:
                            dframe, mf1, mf2, mf3, mf4, mf5, mf6, mf7, mf8 = model(
                                frame, mf1, mf2, mf3, mf4, mf5, mf6, mf7, mf8)
                            loss_sfd += loss_fn(dframe, target)
                            loss_mfd += loss_fn(mf8, target)

                    loss = loss_sfd+loss_mfd

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                print('subepoch = %d, loss = %.4f,loss_sfd = %.4f,loss_mfd = %.4f' % (
                    subepoch + 1, loss.data, loss_sfd.data, loss_mfd.data))  # loss_mfd.data

                loss_sfd_history.append(loss_sfd)
                loss_mfd_history.append(loss_mfd)
                loss_history.append(loss)

                scheduler.step(loss.data)

        print('epoch = %d, loss = %.4f' % (epoch + 1, loss.data))
        if (epoch+1) % save_every == 0:
            Save_modelloss_mfd(model, optimizer, loss_history, loss_sfd_history,
                               loss_mfd_history, name+'%s ' % int(epoch+1))

In [6]:
trainburstserveur(MFDenoiser, paths, loss, optimizer, scheduler, "MFDCnewre",
                  Nb_frames=8, batch_size=13, num_epochs=1000, nb_subepoch=100, save_every=3)

Starting epoch 1 / 1000
subepoch = 1, loss = 0.5582,loss_sfd = 0.2260,loss_mfd = 0.3322
subepoch = 2, loss = 0.5758,loss_sfd = 0.2300,loss_mfd = 0.3458
subepoch = 3, loss = 0.4837,loss_sfd = 0.1934,loss_mfd = 0.2903
subepoch = 4, loss = 0.5022,loss_sfd = 0.2041,loss_mfd = 0.2981
subepoch = 5, loss = 0.5039,loss_sfd = 0.2151,loss_mfd = 0.2888
subepoch = 6, loss = 0.6690,loss_sfd = 0.2985,loss_mfd = 0.3705
subepoch = 7, loss = 0.4842,loss_sfd = 0.1957,loss_mfd = 0.2885
subepoch = 8, loss = 0.4597,loss_sfd = 0.1881,loss_mfd = 0.2715
subepoch = 9, loss = 0.5525,loss_sfd = 0.2500,loss_mfd = 0.3024
subepoch = 10, loss = 0.7063,loss_sfd = 0.3373,loss_mfd = 0.3690
subepoch = 11, loss = 0.5195,loss_sfd = 0.1917,loss_mfd = 0.3278
subepoch = 12, loss = 0.8690,loss_sfd = 0.3823,loss_mfd = 0.4866
subepoch = 13, loss = 0.9140,loss_sfd = 0.4102,loss_mfd = 0.5038
subepoch = 14, loss = 0.5949,loss_sfd = 0.2946,loss_mfd = 0.3003
subepoch = 15, loss = 0.4914,loss_sfd = 0.1959,loss_mfd = 0.2955
subepoch =

subepoch = 51, loss = 0.4351,loss_sfd = 0.1781,loss_mfd = 0.2570
subepoch = 52, loss = 0.7169,loss_sfd = 0.3169,loss_mfd = 0.4000
subepoch = 53, loss = 0.3968,loss_sfd = 0.1736,loss_mfd = 0.2232
subepoch = 54, loss = 0.6075,loss_sfd = 0.2672,loss_mfd = 0.3403
subepoch = 55, loss = 0.6152,loss_sfd = 0.2715,loss_mfd = 0.3437
subepoch = 56, loss = 0.5480,loss_sfd = 0.2396,loss_mfd = 0.3084
subepoch = 57, loss = 0.7220,loss_sfd = 0.3630,loss_mfd = 0.3590
subepoch = 58, loss = 0.4152,loss_sfd = 0.1828,loss_mfd = 0.2324
subepoch = 59, loss = 0.5011,loss_sfd = 0.2283,loss_mfd = 0.2727
subepoch = 60, loss = 0.4379,loss_sfd = 0.1975,loss_mfd = 0.2403
subepoch = 61, loss = 0.4286,loss_sfd = 0.1880,loss_mfd = 0.2406
subepoch = 62, loss = 0.4623,loss_sfd = 0.2000,loss_mfd = 0.2623
subepoch = 63, loss = 0.4179,loss_sfd = 0.1863,loss_mfd = 0.2316
subepoch = 64, loss = 0.5116,loss_sfd = 0.1958,loss_mfd = 0.3158
subepoch = 65, loss = 0.5647,loss_sfd = 0.2506,loss_mfd = 0.3141
subepoch = 66, loss = 0.4

Starting epoch 6 / 1000
subepoch = 1, loss = 0.5297,loss_sfd = 0.2344,loss_mfd = 0.2953
subepoch = 2, loss = 0.4612,loss_sfd = 0.1930,loss_mfd = 0.2682
subepoch = 3, loss = 0.4316,loss_sfd = 0.2035,loss_mfd = 0.2282
subepoch = 4, loss = 0.4403,loss_sfd = 0.2033,loss_mfd = 0.2371
subepoch = 5, loss = 0.6353,loss_sfd = 0.2793,loss_mfd = 0.3561
subepoch = 6, loss = 1.0974,loss_sfd = 0.5711,loss_mfd = 0.5264
subepoch = 7, loss = 0.7259,loss_sfd = 0.3305,loss_mfd = 0.3954
subepoch = 8, loss = 0.4557,loss_sfd = 0.1981,loss_mfd = 0.2576
subepoch = 9, loss = 0.6021,loss_sfd = 0.2562,loss_mfd = 0.3459
subepoch = 10, loss = 0.5508,loss_sfd = 0.2680,loss_mfd = 0.2828
subepoch = 11, loss = 0.4834,loss_sfd = 0.2146,loss_mfd = 0.2688
subepoch = 12, loss = 0.5094,loss_sfd = 0.2312,loss_mfd = 0.2782
subepoch = 13, loss = 0.4776,loss_sfd = 0.2067,loss_mfd = 0.2709
subepoch = 14, loss = 0.4623,loss_sfd = 0.2104,loss_mfd = 0.2518
subepoch = 15, loss = 0.5708,loss_sfd = 0.2508,loss_mfd = 0.3200
subepoch =

subepoch = 51, loss = 0.4850,loss_sfd = 0.2035,loss_mfd = 0.2815
subepoch = 52, loss = 0.4317,loss_sfd = 0.1934,loss_mfd = 0.2383
subepoch = 53, loss = 0.4412,loss_sfd = 0.2035,loss_mfd = 0.2377
subepoch = 54, loss = 0.4051,loss_sfd = 0.1877,loss_mfd = 0.2173
subepoch = 55, loss = 0.4154,loss_sfd = 0.1815,loss_mfd = 0.2338
subepoch = 56, loss = 0.4780,loss_sfd = 0.2166,loss_mfd = 0.2614
subepoch = 57, loss = 0.6247,loss_sfd = 0.3049,loss_mfd = 0.3197
subepoch = 58, loss = 0.5213,loss_sfd = 0.2108,loss_mfd = 0.3105
subepoch = 59, loss = 0.3975,loss_sfd = 0.1841,loss_mfd = 0.2134
subepoch = 60, loss = 0.3987,loss_sfd = 0.1790,loss_mfd = 0.2197
subepoch = 61, loss = 0.5035,loss_sfd = 0.2252,loss_mfd = 0.2783
subepoch = 62, loss = 0.4672,loss_sfd = 0.1976,loss_mfd = 0.2696
subepoch = 63, loss = 0.4675,loss_sfd = 0.2463,loss_mfd = 0.2212
subepoch = 64, loss = 0.4057,loss_sfd = 0.1740,loss_mfd = 0.2316
subepoch = 65, loss = 0.4227,loss_sfd = 0.1953,loss_mfd = 0.2274
subepoch = 66, loss = 0.3

epoch = 10, loss = 0.4523
Starting epoch 11 / 1000
subepoch = 1, loss = 0.4624,loss_sfd = 0.2393,loss_mfd = 0.2231
subepoch = 2, loss = 0.4212,loss_sfd = 0.1973,loss_mfd = 0.2239
subepoch = 3, loss = 0.4961,loss_sfd = 0.2412,loss_mfd = 0.2550
subepoch = 4, loss = 0.5315,loss_sfd = 0.2775,loss_mfd = 0.2540
subepoch = 5, loss = 0.3949,loss_sfd = 0.1811,loss_mfd = 0.2137
subepoch = 6, loss = 0.5127,loss_sfd = 0.2314,loss_mfd = 0.2812
subepoch = 7, loss = 0.3711,loss_sfd = 0.1874,loss_mfd = 0.1837
subepoch = 8, loss = 0.4396,loss_sfd = 0.2155,loss_mfd = 0.2242
subepoch = 9, loss = 0.3680,loss_sfd = 0.1673,loss_mfd = 0.2007
subepoch = 10, loss = 0.5566,loss_sfd = 0.3076,loss_mfd = 0.2490
subepoch = 11, loss = 0.4180,loss_sfd = 0.2016,loss_mfd = 0.2164
subepoch = 12, loss = 0.6266,loss_sfd = 0.2838,loss_mfd = 0.3428
subepoch = 13, loss = 0.5458,loss_sfd = 0.2452,loss_mfd = 0.3006
subepoch = 14, loss = 0.4426,loss_sfd = 0.2095,loss_mfd = 0.2331
subepoch = 15, loss = 0.5315,loss_sfd = 0.2593,l

...

subepoch = 49, loss = 0.4275,loss_sfd = 0.2179,loss_mfd = 0.2095
subepoch = 50, loss = 0.4168,loss_sfd = 0.1962,loss_mfd = 0.2206
subepoch = 51, loss = 0.4869,loss_sfd = 0.2466,loss_mfd = 0.2402
subepoch = 52, loss = 0.4390,loss_sfd = 0.2263,loss_mfd = 0.2127
subepoch = 53, loss = 0.4953,loss_sfd = 0.2467,loss_mfd = 0.2486
subepoch = 54, loss = 0.4135,loss_sfd = 0.2126,loss_mfd = 0.2009
subepoch = 55, loss = 0.4591,loss_sfd = 0.2239,loss_mfd = 0.2353
subepoch = 56, loss = 0.4390,loss_sfd = 0.2027,loss_mfd = 0.2363
subepoch = 57, loss = 0.3899,loss_sfd = 0.1976,loss_mfd = 0.1923
subepoch = 58, loss = 0.4431,loss_sfd = 0.2001,loss_mfd = 0.2429
subepoch = 59, loss = 0.4749,loss_sfd = 0.2391,loss_mfd = 0.2358
subepoch = 60, loss = 0.4474,loss_sfd = 0.2141,loss_mfd = 0.2333
subepoch = 61, loss = 0.3849,loss_sfd = 0.1988,loss_mfd = 0.1862
subepoch = 62, loss = 0.5416,loss_sfd = 0.2846,loss_mfd = 0.2571
subepoch = 63, loss = 0.4476,loss_sfd = 0.2214,loss_mfd = 0.2263
subepoch = 64, loss = 0.8

subepoch = 99, loss = 0.3657,loss_sfd = 0.1747,loss_mfd = 0.1911
subepoch = 100, loss = 0.3481,loss_sfd = 0.1676,loss_mfd = 0.1805
epoch = 20, loss = 0.3481
Starting epoch 21 / 1000
subepoch = 1, loss = 0.4038,loss_sfd = 0.2083,loss_mfd = 0.1955
subepoch = 2, loss = 0.4677,loss_sfd = 0.2231,loss_mfd = 0.2446
subepoch = 3, loss = 0.5225,loss_sfd = 0.2445,loss_mfd = 0.2779
subepoch = 4, loss = 0.4340,loss_sfd = 0.2195,loss_mfd = 0.2145
subepoch = 5, loss = 0.4437,loss_sfd = 0.2377,loss_mfd = 0.2059
subepoch = 6, loss = 0.5435,loss_sfd = 0.2849,loss_mfd = 0.2586
subepoch = 7, loss = 0.3372,loss_sfd = 0.1682,loss_mfd = 0.1690
subepoch = 8, loss = 0.4219,loss_sfd = 0.2138,loss_mfd = 0.2081
subepoch = 9, loss = 0.3870,loss_sfd = 0.1702,loss_mfd = 0.2169
subepoch = 10, loss = 0.4325,loss_sfd = 0.2463,loss_mfd = 0.1861
subepoch = 11, loss = 0.5217,loss_sfd = 0.2818,loss_mfd = 0.2399
subepoch = 12, loss = 0.4154,loss_sfd = 0.2031,loss_mfd = 0.2122
subepoch = 13, loss = 0.3871,loss_sfd = 0.1898,

subepoch = 47, loss = 0.4311,loss_sfd = 0.1910,loss_mfd = 0.2401
subepoch = 48, loss = 0.4665,loss_sfd = 0.2430,loss_mfd = 0.2235
subepoch = 49, loss = 0.5169,loss_sfd = 0.2417,loss_mfd = 0.2752
subepoch = 50, loss = 0.3997,loss_sfd = 0.1903,loss_mfd = 0.2094
subepoch = 51, loss = 0.4113,loss_sfd = 0.1784,loss_mfd = 0.2329
subepoch = 52, loss = 0.4631,loss_sfd = 0.2216,loss_mfd = 0.2415
subepoch = 53, loss = 0.4660,loss_sfd = 0.2295,loss_mfd = 0.2364
subepoch = 54, loss = 0.4087,loss_sfd = 0.1920,loss_mfd = 0.2167
subepoch = 55, loss = 0.4642,loss_sfd = 0.2194,loss_mfd = 0.2448
subepoch = 56, loss = 0.4723,loss_sfd = 0.2407,loss_mfd = 0.2317
subepoch = 57, loss = 0.3940,loss_sfd = 0.1957,loss_mfd = 0.1983
subepoch = 58, loss = 0.4349,loss_sfd = 0.2150,loss_mfd = 0.2199
subepoch = 59, loss = 0.7310,loss_sfd = 0.3437,loss_mfd = 0.3874
subepoch = 60, loss = 0.4781,loss_sfd = 0.2385,loss_mfd = 0.2396
subepoch = 61, loss = 0.4104,loss_sfd = 0.2247,loss_mfd = 0.1857
subepoch = 62, loss = 0.3

subepoch = 97, loss = 0.3977,loss_sfd = 0.1968,loss_mfd = 0.2008
subepoch = 98, loss = 0.4325,loss_sfd = 0.2386,loss_mfd = 0.1939
subepoch = 99, loss = 0.3378,loss_sfd = 0.1677,loss_mfd = 0.1701
subepoch = 100, loss = 0.4018,loss_sfd = 0.2011,loss_mfd = 0.2006
epoch = 25, loss = 0.4018
Starting epoch 26 / 1000
subepoch = 1, loss = 0.3630,loss_sfd = 0.1794,loss_mfd = 0.1837
subepoch = 2, loss = 0.3862,loss_sfd = 0.2097,loss_mfd = 0.1765
subepoch = 3, loss = 0.3691,loss_sfd = 0.1676,loss_mfd = 0.2015
subepoch = 4, loss = 0.4039,loss_sfd = 0.1914,loss_mfd = 0.2125
subepoch = 5, loss = 0.3791,loss_sfd = 0.1847,loss_mfd = 0.1944
subepoch = 6, loss = 0.3613,loss_sfd = 0.1818,loss_mfd = 0.1794
subepoch = 7, loss = 0.3788,loss_sfd = 0.1826,loss_mfd = 0.1962
subepoch = 8, loss = 0.3982,loss_sfd = 0.1963,loss_mfd = 0.2019
subepoch = 9, loss = 0.4319,loss_sfd = 0.2175,loss_mfd = 0.2144
subepoch = 10, loss = 0.4783,loss_sfd = 0.2404,loss_mfd = 0.2379
subepoch = 11, loss = 0.5581,loss_sfd = 0.2725,

KeyboardInterrupt: 

# Test denoiser and save images

In [ ]:
Data_test = T.Compose([
    T.ToTensor()])

testset = ImageFolderRAM('./CBSD68', 0.1, Data_test,
                         Randomnoise=False, loadram=False, loader=RGB_loader)

testloader = torch.utils.data.DataLoader(testset, shuffle=False, batch_size=1)

In [ ]:
filename1 = 'SFD_25SFD_C_NB75000'
filename1 = 'SFD_25SFD_C_NB75000'
Denoiser, optimizer, loss = Load_modelloss(filename1)
Denoiser2, optimizer, loss2 = Load_modelloss(filename2)



In [ ]:
def Show_result(Denoiser, loader, pause, check=False):
    """
Fonction permettant de montrer les resultats de debuitages d'un reseau de neuronnes
Input :
    - model : reseau à tester
    - loader_train : l'ensemble d'image issu du dataloader
    - pause : temps de pause entre chaque image
    - check : if True cela affiche la moyenne de l'image, le min , le max

Affiche :
    -L'image normale
    -L'image bruitée
    -L'image debruité
    -Les PSNR
"""
    Denoiser.eval()

    plt.ion()
    plt.show()

    # Calcul de la PSNR moyenne
    PSNRmoy = 0
    Compteur = 0

    for i, data in enumerate(loader):
        images, groundtrue = data

        plt.figure(1)
        plt.clf()
        plt.imshow(np.clip(np.transpose(tu.make_grid(
            groundtrue, range=(0, 1), nrow=4).numpy(), (1, 2, 0)), 0, 1))
        plt.axis('off')

        print("Image bruitée")
        plt.figure(2)
        plt.clf()
        plt.imshow(np.clip(np.transpose(tu.make_grid(
            images, range=(0, 1), nrow=4).numpy(), (1, 2, 0)), 0, 1))
        plt.axis('off')

        A = psnr(groundtrue.numpy(), images.numpy(), check=check)
        plt.title(r'%f ' % A)

        print("Image débruitée")
        plt.figure(3)
        plt.clf()
        images = Denoiser(
            Variable(images, requires_grad=False).type(gpu_dtype)).data.cpu()
        plt.imshow(np.clip(np.transpose(tu.make_grid(
            images, range=(0, 1), nrow=4).numpy(), (1, 2, 0)), 0, 1))
        plt.axis('off')

        A = psnr(groundtrue.numpy(), images.numpy(), check=check)
        PSNRmoy += A
        Compteur += 1
        plt.title(r'%f ' % A)

        plt.pause(pause)

    print('PSNR moyen = %f ' % (PSNRmoy/Compteur))

In [ ]:
def Show_result2(Denoiser1, Denoiser2, loader, pause, check=False):
    """
Fonction permettant de comparer les resultats de debuitages d'un reseau de neuronnes
Input :
    - Denoiser1 : debruiteur 1 à tester
    - Denoiser2 : debruiteur 2 à tester
    - loader_train : l'ensemble d'image issu du dataloader
    - pause : temps de pause entre chaque image
    - check : if True cela affiche la moyenne de l'image, le min , le max

Affiche :
    - L'image normale
    - L'image bruitée
    - L'image debruité par 1
    - L'image debruité par 2
    - Les PSNR
"""
    Denoiser1.eval()
    Denoiser2.eval()

    plt.ion()
    plt.show()

    # Calcul de la PSNR moyenne
    PSNRmoy1 = 0
    PSNRmoy2 = 0
    Compteur = 0

    for i, data in enumerate(loader):
        images, groundtrue = data

        plt.figure(1)
        plt.clf()
        plt.imshow(np.clip(np.transpose(tu.make_grid(
            groundtrue, range=(0, 1), nrow=4).numpy(), (1, 2, 0)), 0, 1))
        plt.axis('off')

        print("Image bruitée")
        plt.figure(2)
        plt.clf()
        plt.imshow(np.clip(np.transpose(tu.make_grid(
            images, range=(0, 1), nrow=4).numpy(), (1, 2, 0)), 0, 1))
        plt.axis('off')

        A = psnr(groundtrue.numpy(), images.numpy(), check=check)
        plt.title(r'%f ' % A)

        print("Image débruitée par 1")
        plt.figure(3)
        plt.clf()
        images1 = Denoiser1(
            Variable(images, requires_grad=False).type(gpu_dtype)).data.cpu()
        plt.imshow(np.clip(np.transpose(tu.make_grid(
            images1, range=(0, 1), nrow=4).numpy(), (1, 2, 0)), 0, 1))
        plt.axis('off')

        A = psnr(groundtrue.numpy(), images1.numpy(), check=check)
        PSNRmoy1 += A
        Compteur += 1
        plt.title(r' Denoiseur1 PSNR=%f ' % A)

        print("Image débruitée par 2")
        plt.figure(4)
        plt.clf()
        images2 = Denoiser2(
            Variable(images, requires_grad=False).type(gpu_dtype)).data.cpu()
        plt.imshow(np.clip(np.transpose(tu.make_grid(
            images2, range=(0, 1), nrow=4).numpy(), (1, 2, 0)), 0, 1))
        plt.axis('off')

        A = psnr(groundtrue.numpy(), images2.numpy())
        PSNRmoy2 += A
        plt.title(r' Denoiseur2 PSNR= %f ' % A)

        plt.pause(pause)

    print(' Mean PSNR denoiseur1 = %f ' % (PSNRmoy1/Compteur))
    print(' Mean PSNR denoiseur2 = %f ' % (PSNRmoy2/Compteur))

In [ ]:
# For only one denoiser
Show_result(Denoiser, testloader, 0.1, check=False)

In [ ]:
# To compare 2de
Show_result2(Denoiser, Denoiser2, testloader, 0.1, check=False)

In [ ]:
# Loss Plot

##
plt.show()
plt.ion()

##
plt.figure(5)
plt.clf()
plt.semilogy(range(0, len(loss2))[:10000:10], loss2[:10000:10])

plt.figure(6)
plt.clf()
plt.plot(range(0, len(loss))[:10000:10], loss[:10000:10])

##
plt.figure(6)
plt.clf()
plt.plot(range(0, len(loss2))[1::10], loss2[10000::10])


##
plt.figure(7)
plt.clf()
plt.plot(range(0, len(loss3)), loss3)